#Scenario: Employee Work Data for a Tech Company
##Step 1: Prepare Data in PySpark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkSQLExercise1").getOrCreate()

In [2]:
from pyspark.sql import Row
data = [
Row(EmpID=101, Name="Ravi", Department="Engineering", Project="AI Engine",
Salary=95000, HoursPerWeek=42),
Row(EmpID=102, Name="Sneha", Department="Engineering", Project="Data Platform",
Salary=87000, HoursPerWeek=45),
Row(EmpID=103, Name="Kabir", Department="Marketing", Project="Product Launch",
Salary=65000, HoursPerWeek=40),
Row(EmpID=104, Name="Anita", Department="Sales", Project="Client Outreach",
Salary=70000, HoursPerWeek=38),
Row(EmpID=105, Name="Divya", Department="Engineering", Project="AI Engine",
Salary=99000, HoursPerWeek=48),
Row(EmpID=106, Name="Amit", Department="Marketing", Project="Social Media",
Salary=62000, HoursPerWeek=35),
Row(EmpID=107, Name="Priya", Department="HR", Project="Policy Revamp",
Salary=58000, HoursPerWeek=37),
Row(EmpID=108, Name="Manav", Department="Sales", Project="Lead Gen", Salary=73000,
HoursPerWeek=41),
Row(EmpID=109, Name="Neha", Department="Engineering", Project="Security Suite",
Salary=91000, HoursPerWeek=46),
Row(EmpID=110, Name="Farah", Department="HR", Project="Onboarding", Salary=60000,
HoursPerWeek=36)
]
df = spark.createDataFrame(data)
df.show()

+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  106| Amit|  Marketing|   Social Media| 62000|          35|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
|  110|Farah|         HR|     Onboarding| 60000|          36|
+-----+-----+-----------+---------------+------+------------+



#Step 2: Create Views
##Create a Local Temp View

In [3]:
df.createOrReplaceTempView("employees_local")

##Create a Global Temp View

In [4]:
df.createOrReplaceGlobalTempView("employees_global")

#Part A: Exercises on Local View ( employees_local )

In [6]:
# 1. List all employees working on the "AI Engine" project.
spark.sql("select * from employees_local where Project = 'AI Engine'").show()

+-----+-----+-----------+---------+------+------------+
|EmpID| Name| Department|  Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------+------+------------+
|  101| Ravi|Engineering|AI Engine| 95000|          42|
|  105|Divya|Engineering|AI Engine| 99000|          48|
+-----+-----+-----------+---------+------+------------+



In [7]:
# Show all employees from the "Marketing" department with salaries greater than 60,000.
spark.sql("select * from employees_local where Department = 'Marketing' and Salary > 60000").show()

+-----+-----+----------+--------------+------+------------+
|EmpID| Name|Department|       Project|Salary|HoursPerWeek|
+-----+-----+----------+--------------+------+------------+
|  103|Kabir| Marketing|Product Launch| 65000|          40|
|  106| Amit| Marketing|  Social Media| 62000|          35|
+-----+-----+----------+--------------+------+------------+



In [8]:
# 3. Calculate the average salary for each department.
spark.sql("select Department ,  avg(Salary) from employees_local group by Department").show()

+-----------+-----------+
| Department|avg(Salary)|
+-----------+-----------+
|      Sales|    71500.0|
|Engineering|    93000.0|
|  Marketing|    63500.0|
|         HR|    59000.0|
+-----------+-----------+



In [10]:
# 4. List the top 3 highest paid employees overall.
spark.sql("select Name, Salary from employees_local order by Salary Desc limit 3").show()

+-----+------+
| Name|Salary|
+-----+------+
|Divya| 99000|
| Ravi| 95000|
| Neha| 91000|
+-----+------+



In [11]:
# 5. Find employees who work more than 40 hours per week.
spark.sql("select * from employees_local where HoursPerWeek > 40").show()

+-----+-----+-----------+--------------+------+------------+
|EmpID| Name| Department|       Project|Salary|HoursPerWeek|
+-----+-----+-----------+--------------+------+------------+
|  101| Ravi|Engineering|     AI Engine| 95000|          42|
|  102|Sneha|Engineering| Data Platform| 87000|          45|
|  105|Divya|Engineering|     AI Engine| 99000|          48|
|  108|Manav|      Sales|      Lead Gen| 73000|          41|
|  109| Neha|Engineering|Security Suite| 91000|          46|
+-----+-----+-----------+--------------+------+------------+



In [17]:
# 6. Group by project and display the number of employees per project.
spark.sql("select Project, count(*) as no_emp from employees_local group by Project").show()

+---------------+------+
|        Project|no_emp|
+---------------+------+
|  Data Platform|     1|
|      AI Engine|     2|
| Product Launch|     1|
|Client Outreach|     1|
| Security Suite|     1|
|  Policy Revamp|     1|
|       Lead Gen|     1|
|   Social Media|     1|
|     Onboarding|     1|
+---------------+------+



In [24]:
# 7. Drop the local view. Try querying again — what happens?
spark.catalog.dropTempView("employees_local")

True

In [27]:
# error occurs
spark.sql("select * from employees_local ").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `employees_local` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [employees_local], [], false


#Part B: Exercises on Global View ( employees_global )

In [32]:
# 1. Retrieve all "HR" employees working fewer than 38 hours/week.
spark.sql("select * from global_temp.employees_global where Department = 'HR' and HoursPerWeek < 38").show()

+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+



In [34]:
# 2. Calculate the total salary payout for each department.
spark.sql("select Department, sum(Salary) from global_temp.employees_global group by Department").show()

+-----------+-----------+
| Department|sum(Salary)|
+-----------+-----------+
|      Sales|     143000|
|Engineering|     372000|
|  Marketing|     127000|
|         HR|     118000|
+-----------+-----------+



In [36]:
# 3. For each employee, add a derived column Status :
# If HoursPerWeek > 45 → 'Overworked'
# Otherwise → 'Normal'
from pyspark.sql.functions import when
df_status = df.withColumn('Status', when(df.HoursPerWeek > 45, 'Overworked').otherwise('Normal'))
df_status.show()

+-----+-----+-----------+---------------+------+------------+----------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|    Status|
+-----+-----+-----------+---------------+------+------------+----------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|    Normal|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|    Normal|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|    Normal|
|  104|Anita|      Sales|Client Outreach| 70000|          38|    Normal|
|  105|Divya|Engineering|      AI Engine| 99000|          48|Overworked|
|  106| Amit|  Marketing|   Social Media| 62000|          35|    Normal|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|    Normal|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|    Normal|
|  109| Neha|Engineering| Security Suite| 91000|          46|Overworked|
|  110|Farah|         HR|     Onboarding| 60000|          36|    Normal|
+-----+-----+-----------+---------------+------+---

In [37]:
# 4. Count the total number of employees working on each "Project" .
spark.sql("select Project, count(*) as no_emp from global_temp.employees_global group by Project").show()

+---------------+------+
|        Project|no_emp|
+---------------+------+
|  Data Platform|     1|
|      AI Engine|     2|
| Product Launch|     1|
|Client Outreach|     1|
| Security Suite|     1|
|  Policy Revamp|     1|
|       Lead Gen|     1|
|   Social Media|     1|
|     Onboarding|     1|
+---------------+------+



In [40]:
# 5. List employees whose salary is above the average salary in their department.
from pyspark.sql.functions import col
avg_salary = df.groupBy("Department").avg("Salary").withColumnRenamed("avg(Salary)", "AvgSalary")
result = df.join(avg_salary, "Department").filter(col("Salary") > col("AvgSalary")).select("EmpID", "Name", "Department", "Salary")
result.show()

+-----+-----+-----------+------+
|EmpID| Name| Department|Salary|
+-----+-----+-----------+------+
|  101| Ravi|Engineering| 95000|
|  105|Divya|Engineering| 99000|
|  103|Kabir|  Marketing| 65000|
|  108|Manav|      Sales| 73000|
|  110|Farah|         HR| 60000|
+-----+-----+-----------+------+



In [41]:
# 6. Open a new Spark session and query "global_temp.employees_global" from there.
new_spark = SparkSession.builder.appName("SparkSQL_new").getOrCreate()

In [42]:
spark.sql("select * from global_temp.employees_global ").show()

+-----+-----+-----------+---------------+------+------------+
|EmpID| Name| Department|        Project|Salary|HoursPerWeek|
+-----+-----+-----------+---------------+------+------------+
|  101| Ravi|Engineering|      AI Engine| 95000|          42|
|  102|Sneha|Engineering|  Data Platform| 87000|          45|
|  103|Kabir|  Marketing| Product Launch| 65000|          40|
|  104|Anita|      Sales|Client Outreach| 70000|          38|
|  105|Divya|Engineering|      AI Engine| 99000|          48|
|  106| Amit|  Marketing|   Social Media| 62000|          35|
|  107|Priya|         HR|  Policy Revamp| 58000|          37|
|  108|Manav|      Sales|       Lead Gen| 73000|          41|
|  109| Neha|Engineering| Security Suite| 91000|          46|
|  110|Farah|         HR|     Onboarding| 60000|          36|
+-----+-----+-----------+---------------+------+------------+



#Bonus Challenges

In [43]:
# 1. Use a window function to assign rank to employees within each department based on salary.
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, desc
windowSpec = Window.partitionBy("Department").orderBy(desc("Salary"))
ranked_df = df.withColumn("Rank", rank().over(windowSpec))
ranked_df.select("EmpID", "Name", "Department", "Salary", "Rank").show()

+-----+-----+-----------+------+----+
|EmpID| Name| Department|Salary|Rank|
+-----+-----+-----------+------+----+
|  105|Divya|Engineering| 99000|   1|
|  101| Ravi|Engineering| 95000|   2|
|  109| Neha|Engineering| 91000|   3|
|  102|Sneha|Engineering| 87000|   4|
|  110|Farah|         HR| 60000|   1|
|  107|Priya|         HR| 58000|   2|
|  103|Kabir|  Marketing| 65000|   1|
|  106| Amit|  Marketing| 62000|   2|
|  108|Manav|      Sales| 73000|   1|
|  104|Anita|      Sales| 70000|   2|
+-----+-----+-----------+------+----+



In [44]:
# 2. Create another view (local or global) that only contains "Engineering" employees.
df.filter(df.Department == "Engineering").createOrReplaceGlobalTempView("engineering_employees_global")

In [49]:
# 3. Create a SQL view that filters out all employees working < 38 hours and saves it as "active_employees"
active_df = df.filter(df.HoursPerWeek < 38)
active_df.createOrReplaceTempView("active_employees")
spark.sql("SELECT * FROM active_employees").show()

+-----+-----+----------+-------------+------+------------+
|EmpID| Name|Department|      Project|Salary|HoursPerWeek|
+-----+-----+----------+-------------+------+------------+
|  106| Amit| Marketing| Social Media| 62000|          35|
|  107|Priya|        HR|Policy Revamp| 58000|          37|
|  110|Farah|        HR|   Onboarding| 60000|          36|
+-----+-----+----------+-------------+------+------------+

